# POC mesures de pertinence
Projet Interpromo 2020, groupe 2,
Corentin Prat-Marca

Définition de score de pertinences pour chaque sources/sites afin de les classer et définir une priorité pour le crawl et le scrap

In [11]:
import nltk   #the importations for cleandesc
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer 
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('french') #we're working on french articles
import pandas as pd


def cleandesc(desc : str): #function to remove stopwords and suffixes
  """Doucumentation
  @Sonia Bezombes
  """
  sent = desc
  sent = "".join([x.lower() if x.isalpha()  else " " for x in sent])
  Porter=SnowballStemmer('french')
  sent = " ".join([Porter.stem(x) if x.lower() not in stop_words  else "" for x in sent.split()])
  sent = " ".join(sent.split())
  return sent

def calcul_score(texte : str,lexique : list): #function to count the number of words of a list in a text
  """Documentation
  Parameters:
    texte : the text we're working on
    lexique : the list of words we're looking for
  Out :
    somme : the number of lexique's words we found in texte
  """
  somme : int=0
  for mot in lexique:
    somme += texte.count(' '+mot+' ')
  return somme


#before using pertinence 4, we have to clean the columns we're gonna work on like that :
#df_scrapped_articles['column_name']= [cleandesc(x.column_name) for x in df_scrapped_articles.itertuples()]
#cleaning art_content all the time, just once
#df_scrapped_articles['art_content']= [cleandesc(x.art_content) for x in df_work.itertuples()]

def pertinence_4(df : pd.DataFrame , list_columns : list, lexiqueI : list, lexiqueG : list):
  """Documentation
  Parameters:
    df : a data frame of scrapped articles. It must contains a "art_content" columns to determine its size
    list_columns : the list of columns we want to count the words of the two lexiques on. 
    lexiqueI : the innovation lexique, it has to be a list
    lexiqueG : the gestion lexique, it has to be a list
  Out :
    df : the same data but with the scores and the step to have it
  """
  list_nom_colonne_I : list = [] #the list of the names of the innovation's score columns 
  list_nom_colonne_G : list = [] #the list of the names of the gestion's score columns 
  cleanedG : list = [cleandesc(mot) for mot in lexiqueG] #cleaning the gestion's lexique
  cleanedI : list = [cleandesc(mot) for mot in lexiqueI] #cleaning the gestion's lexique

  for columns in list_columns:

    nomI : str ='score_innovation_'+columns #the name of the new columns : score_innovation_column_name or score_gestion_column_name
    nomG : str ='score_gestion_'+columns
    df[nomI]=[calcul_score(df.loc[i][columns], cleanedI) for i in range(len(df))] #creating the new columns using calcul_score, counting the number of words
    df[nomG]=[calcul_score(df.loc[i][columns], cleanedG) for i in range(len(df))]
    df['score_'+columns]=[df.loc[i][nomI]+df.loc[i][nomG] for i in range(len(df))] #new column, wich is the sum of gestion and innovation scores
    list_nom_colonne_I.append(nomI)
    list_nom_colonne_G.append(nomG)
  
  df['taille_content'] = [len(df['art_content'][i]) for i in range(len(df))] #length of the cleaned content of the article
  df['score_I'] = df[list_nom_colonne_I].sum(axis = 1) #sum of all the innovation's scores
  df['score_G'] = df[list_nom_colonne_G].sum(axis = 1) #sum of all the gestion's scores
  df['score_abs'] = abs(df['score_I']-df['score_G']) #absolute value of the difference of the number of gestion's words and innovation's words
  df['detct_0'] = [0 if (x.score_I == 0) or (x.score_G == 0) else 1 for x in df.itertuples()] #finding if there is 0 gestion's words or 0 innovation's words, to penalise the articles which don't speak of innovation or gestion
  df['score_total'] = [(1.09*(x.score_I+x.score_G)-abs(x.score_I-x.score_G))*100/x.taille_content if x.detct_0 == 0 else (1.09*(x.score_I+x.score_G)-abs(x.score_I-x.score_G))*1000/x.taille_content for x in df.itertuples()]
  
  return df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


On cherche à attribuer un score à des articles scrappés en regardant combien on trouve de mots de chaque lexique. On choisit sur quels parties on se base (nom de l'article, contenu, etc), et on calcule un score dépendant du nombre de mots des lexiques (plus il y a de mots mieux c'est, si un article comporte 0 mots d'un lexique on pénalise) de la différence des nombres de mots de chaque lexique (on veut à peu près autant de mots de chaque lexique), et de la taille de l'article.

In [12]:
import re
def url_site(url_art : str):
  """Documentation
  @Flavien Caminade et Corentin Prat-Marca
  Parameters:
    url_art : the url of an article
  Out :
    url_site : the url of a site obtained with the article's url
    """
  site : list  =url_art.split("://")
  #keep everything on the right side of "https://""
  if site[0]=="https" or site[0]=="http":
      url_site : str = site[1]
  else:
      url_site : str = site[0]
  #delete everything after the first "/"
  tab : list = url_site.split("/")
  url_site = tab[0]
  return url_site

def pertinence_4_source(df_pertinence_4 : pd.DataFrame):
  """Documentation
  Parameters:
    df_pertinence_4 : a data frame of scrapped articles with their score
  Out :
    df : a data frame with the score for each sites
  """
  dico : dict ={} #dictionnary : keys=sources url, values=sum of the scores of each articles
  for articles in df_calcul_global.itertuples():

    if url_site(articles.art_url) in dico: #if the source il already in the dictionnary
      dico[url_site(articles.art_url)][0] += (articles.score_total)**2 #using square so higher score have more values
      dico[url_site(articles.art_url)][1] += 1 #increasing the number of articles for the source
    else :
      dico[url_site(articles.art_url)] = [(articles.score_total)**2, 1]

  df : pd.DataFrame = pd.DataFrame.from_dict(dico, orient = 'index', columns = ['score_total', 'nb_articles'])
  df['score_moyen'] = df.apply(lambda row: row.score_total / row.nb_articles, axis = 1) #mean of the scores of the artciles for each sources
  
  return df
#pertinence_4_source(df_pertinence_4)

On utilise les scores des articles pour obtenir un score pour chaque source en faisant la moyenne. Pour cela on utilise une fonction qui à partir d'un url d'article trouve l'url du site source. Du coup, on se retrouve avec linkedin.com=/=fr.linkedin, mais 2 sites différents avec le même nom (site.fr, www.site.com, etc) sont bien différenciés. Ca nous permet aussi de faire la différence entre la version française et anglaise d'un site.

In [7]:
def pertinence_6(df : pd.DataFrame): #df avec proba innovation et proba gestion
  """Documentation
  Parameters:
    df : a data frame of scrapped articles with the probability for each to contain a "gestion" word or an "innovation" word (group 5 work)
  Out :
    df : the same data frame with 4 more columns each containing a score based on the probability
    """
  df['pertinence_10'] = df.apply(lambda row: (10*(row.probaInnovation + row.probaGestion) - abs(row.probaInnovation-row.probaGestion))/20, axis = 1) #(10*(motsGestion+motsInnovation)-abs(motsGestion-motsInnovation)) / 20 (pour un score entre 0 et 1)
  df['pertinence_5'] = df.apply(lambda row: (5*(row.probaInnovation + row.probaGestion) - abs(row.probaInnovation-row.probaGestion))/10, axis = 1) #(5*(motsGestion+motsInnovation)-abs(motsGestion-motsInnovation)) / 10 (pour un score entre 0 et 1)
  df['pertinence_2'] = df.apply(lambda row: (2*(row.probaInnovation + row.probaGestion) - abs(row.probaInnovation-row.probaGestion))/4, axis = 1) #(2*(motsGestion+motsInnovation)-abs(motsGestion-motsInnovation)) / 4 (pour un score entre 0 et 1)
  df['pertinence_F1'] = df.apply(lambda row: 2*(row.probaInnovation * row.probaGestion) / (row.probaInnovation+row.probaGestion), axis = 1) #F1 score adapté
  
  return df

Cette fonction permet de créer un score de pertinance à partir d'un df d'articles scrappés dont on a les probabilités d'apparitions d'un mot de gestions ou d'innovation (travail du groupe 5).
Elle permet d'évaluer les articles pour avoir une idée de la pertinence. Les scores vont de 0 à 1, et prennent en compte la somme des probabilités (plus on a de mots des lexiques, mieux c'est) et la différence des probabilités (les sites avec un nombre à peu près egal de mots de chaques lexiques sont plus intéressants).

In [14]:
def pertinence_6_source(df_pertinence_6 : pd.DataFrame):
  """Documentation
  Parameters: 
    df_pertinence_6 : a data frame of scrapped articles with their score, obtained with the function pertinence_6
  Out :
    df : a data frame of the means of each score for each source
    """
  dico_pertinence : dict = {} #key : a source, values : a list [score_10,score_5,score_2,score_F1,number of articles]
  for articles in df_pertinence_6.itertuples():

    if url_site(articles.art_url) in dico_pertinence: #if the source arlready is in the dictionnary
      dico_pertinence[url_site(articles.art_url)][0] += articles.pertinence_10**2 #using square function so that articles with a higher score are more important
      dico_pertinence[url_site(articles.art_url)][1] += articles.pertinence_5**2
      dico_pertinence[url_site(articles.art_url)][2] += articles.pertinence_2**2
      dico_pertinence[url_site(articles.art_url)][3] += articles.pertinence_F1**2
      dico_pertinence[url_site(articles.art_url)][4] += 1 #increasing the number or articles
    else :
      dico_pertinence[url_site(articles.art_url)]=[articles.pertinence_10**2,articles.pertinence_5**2,articles.pertinence_2**2,articles.pertinence_F1**2,1] #adding a new source
  
  df : pd.DataFrame = pd.DataFrame.from_dict(dico_pertinence, orient = 'index', columns = ['pertinence_10','pertinence_5','pertinence_2','pertinence_F1', 'nb_articles']) #turning the dictionnary into a data frame
  df['pertinence_10_moyenne'] = df.apply(lambda row: row.pertinence_10 / row.nb_articles, axis = 1)
  df['pertinence_5_moyenne'] = df.apply(lambda row: row.pertinence_5 / row.nb_articles, axis = 1)
  df['pertinence_2_moyenne'] = df.apply(lambda row: row.pertinence_2 / row.nb_articles, axis = 1)
  df['pertinence_F1_moyenne'] = df.apply(lambda row: row.pertinence_F1 / row.nb_articles, axis = 1)
  
  return df

On cherche à obtenir la pertinence des sites, donc on fait la moyenne de la pertinence de chaque articles des sites obtenue avec la fonction précédente. On utilise la fonction carré pour que les articles avec un score faible soient moins importants dans le score total que ceux avec un score élevé.

En résumé, on a 2 manières d'obtenir des scores de pertinence qu'il peut être intéressant de combiner. Par contre elles sont très dépendantes des lexiques et les synonymes ne sont pas forcément pris en compte.